In [ ]:
!pip3 install numpy pillow torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [31]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [32]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [33]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=2, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
image , label = train_data[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [35]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [36]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3 , 12 , 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12 , 24 , 5)
        self.fc1 = nn.Linear(24 * 5 * 5 , 120)
        self.fc2 = nn.Linear(120 , 84)
        self.fc3 = nn.Linear(84 , 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x , 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [37]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters() , lr=0.001 , momentum=0.9)


In [38]:
for epoch in range(15):
    print(f'Training epoch {epoch + 1}...')

    running_loss = 0.0

    for i , data in enumerate(train_loader):
        inputs , labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs , labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Loss: {running_loss/len(train_loader):.4f}')

Training epoch 1...
Loss: 1.5876
Training epoch 2...
Loss: 1.2178
Training epoch 3...
Loss: 1.0858
Training epoch 4...
Loss: 1.0100
Training epoch 5...
Loss: 0.9498
Training epoch 6...
Loss: 0.9179
Training epoch 7...
Loss: 0.8781
Training epoch 8...
Loss: 0.8570
Training epoch 9...
Loss: 0.8360
Training epoch 10...
Loss: 0.8182
Training epoch 11...
Loss: 0.8073
Training epoch 12...
Loss: 0.8035
Training epoch 13...
Loss: 0.7847
Training epoch 14...
Loss: 0.7828
Training epoch 15...
Loss: 0.7741


In [39]:
torch.save(net.state_dict() , 'trained_net.pth')

In [40]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<ipython-input-40-c299e2fffacc>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [41]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images , labels = data
        outputs = net(images)
        _ , predicted = torch.max(outputs , 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network: {accuracy} %')

Accuracy of the network: 63.78 %


In [45]:
new_transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['example1.jpg', 'example2.jpg', 'example3.jpg', 'example4.jpg' , 'example5.jpg', 'example6.jpg', 'example7.jfif', 'example8.jpg', 'example9.jpg', 'example10.jpg' ,'example11.jpeg']
images = [load_image(path) for path in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _ , protected = torch.max(output , 1)
    print(f'Prediction: {class_names[protected.item()]}')

Prediction: dog
Prediction: plane
Prediction: truck
Prediction: plane
Prediction: car
Prediction: bird
Prediction: dog
Prediction: deer
Prediction: frog
Prediction: ship
Prediction: deer


1. Dog
2. Dog
3. truck
4. Plane
5. Car
6. bird
7. Cat
8. Deer
9. Frog
10. ship
11. Horse